In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [8]:
df = pd.read_csv('/Users/sanjayroberts1/Desktop/data_science_tools1/sudeste.csv')

In [9]:
df = df.set_index('mdct')
df.index = pd.to_datetime(df.index)

In [8]:
d = {}
for col in df:
    d[col] = df[col].unique().tolist()

In [21]:
#print(d['temp'])

In [25]:
df.shape

(9779168, 30)

In [20]:
print(df.isna().sum())

wsid          0
wsnm          0
elvt          0
lat           0
lon           0
inme          0
city          0
prov          0
date          0
yr            0
mo            0
da            0
hr            0
prcp    8371184
stp           0
smax          0
smin          0
gbrd    4108820
temp         31
dewp        475
tmax         26
dmax        310
tmin         34
dmin        807
hmdy          0
hmax         12
hmin         44
wdsp     925561
wdct          0
gust     316474
dtype: int64


Some stations do not have wind or rain sensors, owing to why there are so many Null values. Will set them to zero to be able to do cleanup, then separate stations to do individual analysis

Gbrd has null values for when the sun is down (little less than half the time which makes sense). Make these zero values also to not affect analysis.

Other column null values we can set to zero and interpolate to get a value.

- Separate data for each station
- Each station may have it's own issues for data cleanup, so we separate and apply a cleanup function for each

In [22]:
df['prcp'].fillna(0,inplace=True)

In [31]:
df['gbrd'].fillna(0,inplace=True)

In [33]:
df.fillna(0,inplace=True)

In [34]:
df.isna().sum()

wsid    0
wsnm    0
elvt    0
lat     0
lon     0
inme    0
city    0
prov    0
date    0
yr      0
mo      0
da      0
hr      0
prcp    0
stp     0
smax    0
smin    0
gbrd    0
temp    0
dewp    0
tmax    0
dmax    0
tmin    0
dmin    0
hmdy    0
hmax    0
hmin    0
wdsp    0
wdct    0
gust    0
dtype: int64

Drop where all sensor columns are 0

In [36]:
col = ['prcp', 'stp', 'smax', 'smin', 'gbrd', 'temp',
       'dewp', 'tmax', 'dmax', 'tmin', 'dmin', 'hmdy', 'hmax', 'hmin', 'wdsp',
       'wdct', 'gust']
df = df[(df[col] != 0).any(axis=1)]


In [37]:
df.shape

(9159199, 30)

9779168 - 9159199 = 619969 columns dropped

take care of all but wind data. Look closer into wind data issues

In [39]:
df['temp'].replace(0, np.nan, inplace=True)
df['temp'].interpolate('time', inplace=True, limit_direction='both')
df['dewp'].replace(0, np.nan, inplace=True)
df['dewp'].interpolate('time', inplace=True, limit_direction='both')
df['tmax'].replace(0, np.nan, inplace=True)
df['tmax'].interpolate('time', inplace=True, limit_direction='both')
df['tmin'].replace(0, np.nan, inplace=True)
df['tmin'].interpolate('time', inplace=True, limit_direction='both')
df['dmax'].replace(0, np.nan, inplace=True)
df['dmax'].interpolate('time', inplace=True, limit_direction='both')
df['dmin'].replace(0, np.nan, inplace=True)
df['dmin'].interpolate('time', inplace=True, limit_direction='both')
df['hmax'].replace(0, np.nan, inplace=True)
df['hmax'].interpolate('time', inplace=True, limit_direction='both')
df['hmin'].replace(0, np.nan, inplace=True)
df['hmin'].interpolate('time', inplace=True, limit_direction='both')

Count number of nonzeros in each column

In [40]:
df.astype(bool).sum(axis=0)

wsid    9159199
wsnm    9159199
elvt    9119489
lat     9119489
lon     9119489
inme    9159199
city    9159199
prov    9159199
date    9159199
yr      9159199
mo      9159199
da      9159199
hr      8778203
prcp     661305
stp     9077445
smax    9073730
smin    9073810
gbrd    4790858
temp    9159199
dewp    9159198
tmax    9159199
dmax    9159199
tmin    9159199
dmin    9159199
hmdy    9099495
hmax    9159199
hmin    9159199
wdsp    8025423
wdct    8908786
gust    8621514
dtype: int64

Count number of zeros in each column

In [44]:
(df == 0).astype(int).sum(axis=0)

wsid          0
wsnm          0
elvt      39710
lat       39710
lon       39710
inme          0
city          0
prov          0
date          0
yr            0
mo            0
da            0
hr       380996
prcp    8497894
stp       81754
smax      85469
smin      85389
gbrd    4368341
temp          0
dewp          1
tmax          0
dmax          0
tmin          0
dmin          0
hmdy          0
hmax          0
hmin          0
wdsp    1133776
wdct     250413
gust     537685
dtype: int64

In [43]:
df['hmdy'].replace(0, np.nan, inplace=True)
df['hmdy'].interpolate('time', inplace=True, limit_direction='both')

look into why elevation, lat, lon is zero (all same weather station?), same with smin, smax etc, and look at surrornding values to see if it makes sense

hr, prcp, and gbrd are ok to be 0

https://stackoverflow.com/questions/40660088/get-first-row-of-dataframe-in-python-pandas-based-on-criteria

then work on wind stuff to clean that data up

In [22]:
df.to_hdf('cleaning_up_df.hdf','mydata',mode='w')

In [4]:
df = pd.read_hdf('cleaning_up_df.hdf','mydata')

In [48]:
df[df.elvt == 0].index[0]

Timestamp('2008-07-25 18:00:00')

In [50]:
df[df.elvt == 0].head()

,wsid,wsnm,elvt,lat,lon,inme,city,prov,date,yr,mo,da,hr,prcp,stp,smax,smin,gbrd,temp,dewp,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
mdct,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2008-07-25 18:00:00,418,MOELA,0.0,0.0,0.0,ZZZ3,Guarujá,SP,2008-07-25,2008,7,25,18,0.0,1014.3,1014.3,1014.0,426.700,20.0,16.0,20.2,16.9,19.8,15.8,78.0,83.0,76.0,3.3,242.0,7.2
2008-07-25 19:00:00,418,MOELA,0.0,0.0,0.0,ZZZ3,Guarujá,SP,2008-07-25,2008,7,25,19,0.0,1014.8,1014.8,1014.2,167.094,19.9,15.3,20.0,16.3,19.8,15.3,75.0,80.0,75.0,2.5,239.0,7.4
2008-07-25 20:00:00,418,MOELA,0.0,0.0,0.0,ZZZ3,Guarujá,SP,2008-07-25,2008,7,25,20,0.0,1015.2,1015.2,1014.8,95.589,19.6,15.4,19.9,15.7,19.5,15.2,77.0,78.0,74.0,2.8,222.0,6.8
2008-07-25 21:00:00,418,MOELA,0.0,0.0,0.0,ZZZ3,Guarujá,SP,2008-07-25,2008,7,25,21,0.0,1015.8,1015.8,1015.2,12.199,19.3,15.4,19.6,15.9,19.2,15.2,78.0,80.0,77.0,1.9,276.0,8.0
2008-07-25 22:00:00,418,MOELA,0.0,0.0,0.0,ZZZ3,Guarujá,SP,2008-07-25,2008,7,25,22,0.0,1016.7,1016.7,1015.8,0.000,19.2,14.0,19.3,15.5,19.2,13.7,72.0,79.0,70.0,4.0,94.0,8.4


In [52]:
df[df.elvt == 0].count()

wsid    39710
wsnm    39710
elvt    39710
lat     39710
lon     39710
inme    39710
city    39710
prov    39710
date    39710
yr      39710
mo      39710
da      39710
hr      39710
prcp    39710
stp     39710
smax    39710
smin    39710
gbrd    39710
temp    39710
dewp    39710
tmax    39710
dmax    39710
tmin    39710
dmin    39710
hmdy    39710
hmax    39710
hmin    39710
wdsp    39710
wdct    39710
gust    39710
dtype: int64

This explains the 0 values. Maybe the metadata for the station did not get input correctly? Let's go to the internet and see what the average lat/lon and elevation is for Guarujá

https://en.wikipedia.org/wiki/Guaruj%C3%A1

It is a very small area (55 sq miles)
Elevation is 4 meters
lat lon is 23°59′37″S 46°15′23″W --> lat -23.993611, lon -46.256389
We can change this in our dataset to get a more accurate description

In [5]:
df['elvt'].replace(0, 4, inplace=True)
df['lat'].replace(0, -23.993611, inplace=True)
df['lon'].replace(0, -46.256389, inplace=True)

Let's look at what needs cleaning next

In [6]:
(df == 0).astype(int).sum(axis=0)

wsid          0
wsnm          0
elvt          0
lat           0
lon           0
inme          0
city          0
prov          0
date          0
yr            0
mo            0
da            0
hr       380996
prcp    8497894
stp       81754
smax      85469
smin      85389
gbrd    4368341
temp          0
dewp          1
tmax          0
dmax          0
tmin          0
dmin          0
hmdy          0
hmax          0
hmin          0
wdsp    1133776
wdct     250413
gust     537685
dtype: int64

Let's look at stp. It looks like a bunch of the stations just use the same Smax value as they do for stp...

In [13]:
stp_smax_equal = df[df['stp'] == df['smax']].count().stp
stp_smax_equal

3652525

3652525/9159199 = 0.398, so about 40% of the data has this issue... Does this warrant dropping one of the columns? Or is smax a good enough 

Uhoh, then the other 40% has stp = smin

In [16]:
stp_smin_equal = df[df['stp'] == df['smin']].count().stp
stp_smin_equal

3458158

What about when smax = smin?

In [17]:
smax_smin_equal = df[df['smax'] == df['smin']].count().stp
smax_smin_equal

92853

9159199 - (3652525 + 3458158 + 92853) = 1955663 rows have independent values for all 3.

How much of our analysis do we want to use stp, smin, smax? Or drop? Or go thru for each station to see the issues and cleanup individually?

In [24]:
df[df.wdct == 0].head(300)

,wsid,wsnm,elvt,lat,lon,inme,city,prov,date,yr,mo,da,hr,prcp,stp,smax,smin,gbrd,temp,dewp,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
mdct,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2008-05-02 22:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2008-05-02,2008,5,2,22,0.0,982.5,0.0,0.0,0.000,24.1,23.2,21.700000,17.200000,21.100000,15.800000,95.0,79.0,90.000000,0.0,0.0,0.0
2008-05-07 04:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2008-05-07,2008,5,7,4,0.0,984.4,0.0,0.0,0.000,22.4,21.9,21.800000,5.300000,18.800000,5.200000,97.0,87.0,86.000000,0.0,0.0,0.0
2008-05-08 06:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2008-05-08,2008,5,8,6,0.0,983.8,0.0,0.0,0.000,22.1,21.5,17.000000,7.800000,15.600000,3.100000,96.0,75.0,67.000000,0.0,0.0,0.0
2008-05-14 09:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2008-05-14,2008,5,14,9,0.0,987.2,0.0,0.0,4.607,19.1,18.6,15.100000,13.200000,14.700000,10.900000,97.0,85.0,96.000000,0.0,0.0,0.0
2008-05-17 00:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2008-05-17,2008,5,17,0,0.0,988.1,0.0,0.0,0.000,23.4,22.8,21.100000,13.600000,20.100000,11.300000,96.0,76.0,88.000000,0.0,0.0,0.0
2008-05-19 23:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2008-05-19,2008,5,19,23,0.0,986.3,0.0,0.0,0.000,24.4,22.4,20.600000,12.000000,19.700000,11.700000,89.0,77.0,67.000000,0.0,0.0,0.0
2011-09-27 12:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2011-09-27,2011,9,27,12,0.0,990.6,0.0,0.0,203.514,28.0,16.1,23.700000,12.300000,14.600000,11.000000,48.0,80.0,71.000000,0.0,0.0,0.0
2007-01-25 03:00:00,303,VITÓRIA,9.0,-20.271094,-40.306069,A612,Vitória,ES,2007-01-25,2007,1,25,3,0.0,1014.6,1015.1,1014.6,0.000,24.0,23.4,24.100000,23.400000,24.000000,23.300000,96.0,96.0,96.000000,0.2,0.0,0.9
2008-06-08 01:00:00,303,VITÓRIA,9.0,-20.271094,-40.306069,A612,Vitória,ES,2008-06-08,2008,6,8,1,0.0,1019.6,1019.6,1019.4,0.000,20.2,19.2,21.200000,20.000000,20.200000,19.200000,94.0,94.0,92.000000,0.1,0.0,1.2


Looks like some stations (178) did not capture wind gust data, though they have wind direction and speed. Then some stations do not capture all 3 measurements

In [25]:
df[df.gust == 0].head(300)

,wsid,wsnm,elvt,lat,lon,inme,city,prov,date,yr,mo,da,hr,prcp,stp,smax,smin,gbrd,temp,dewp,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
mdct,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2007-11-06 09:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06,2007,11,6,9,0.0,986.7,986.7,985.7,214.149,22.9,18.3,22.9,18.3,18.2,17.100000,75.0,94.0,75.0,0.0,248.0,0.0
2007-11-14 10:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-14,2007,11,14,10,0.0,986.9,986.9,986.6,884.623,26.2,17.9,26.4,18.0,23.9,17.500000,60.0,69.0,59.0,3.1,109.0,0.0
2007-11-17 10:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-17,2007,11,17,10,0.0,986.9,986.9,986.4,850.989,26.4,16.9,26.9,17.3,25.5,16.800000,56.0,59.0,55.0,4.5,139.0,0.0
2007-11-18 10:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-18,2007,11,18,10,0.0,986.6,986.6,985.9,800.069,25.7,17.6,26.0,17.8,24.7,17.400000,61.0,65.0,60.0,3.5,141.0,0.0
2007-11-21 10:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-21,2007,11,21,10,0.0,986.6,986.7,986.3,833.818,26.9,19.4,26.9,20.1,25.3,19.400000,64.0,72.0,64.0,2.7,90.0,0.0
2007-11-22 10:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-22,2007,11,22,10,0.0,987.0,987.0,986.6,809.446,26.1,18.3,26.5,18.5,25.5,17.900000,62.0,64.0,60.0,4.0,105.0,0.0
2007-11-22 22:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-22,2007,11,22,22,0.0,982.6,982.6,982.1,0.000,24.6,20.4,26.3,21.0,24.6,19.900000,78.0,80.0,68.0,0.0,141.0,0.0
2007-11-23 22:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-23,2007,11,23,22,0.0,982.2,982.2,981.3,0.000,25.9,22.1,27.4,22.4,25.8,20.900000,80.0,80.0,69.0,0.0,137.0,0.0
2007-11-26 10:00:00,178,SÃO GONÇALO,237.0,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-26,2007,11,26,10,0.0,985.5,985.5,985.0,822.372,26.0,17.3,26.0,18.0,24.7,17.200000,59.0,67.0,59.0,2.9,107.0,0.0


In [31]:
len(df.city.unique())

117

Split station and data and store in a dict. Then we can individually clean each set?

In [26]:
individual_stations = {}
for each in df.wsid.unique():
    is_ = df['wsid'] == each
    k = str(each)
    individual_stations[k] = df[is_]

In [27]:
individual_stations.keys()

dict_keys(['178', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '322', '323', '324', '325', '326', '327', '328', '329', '330', '331', '332', '333', '334', '335', '336', '337', '338', '339', '340', '341', '342', '343', '344', '345', '346', '347', '348', '349', '350', '351', '352', '353', '354', '355', '356', '357', '358', '359', '360', '361', '362', '363', '364', '365', '366', '367', '368', '369', '370', '371', '372', '373', '374', '375', '376', '377', '378', '379', '380', '381', '382', '383', '384', '385', '386', '387', '388', '390', '391', '392', '393', '394', '395', '396', '397', '398', '399', '400', '401', '402', '403', '404', '405', '406', '407', '408', '409', '410', '411', '412', '413', '414', '415', '416', '417', '418', '419', '420', '421', '422', '423'])